In [1]:
!git clone https://github.com/CS726-AdvML/long_range_TSF.git

Cloning into 'long_range_TSF'...
remote: Enumerating objects: 340, done.
remote: Counting objects: 100% (340/340), done.
remote: Compressing objects: 100% (246/246), done.
remote: Total 340 (delta 69), reused 325 (delta 56), pack-reused 0
Receiving objects: 100% (340/340), 19.63 MiB | 32.26 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [1]:
cd /content/long_range_TSF/TFT_NBEATS

/content/long_range_TSF/TFT_NBEATS


In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# !set -e

# !pip3 install virtualenv # Assumes pip3 is installed!
# !python3 -m virtualenv $OUTPUT_FOLDER/venv
# !source $OUTPUT_FOLDER/venv/bin/activate
# !pip3 install --pre "tensorflow==1.15.*"
# !pip3 install -r requirements.txt

# !python3 -m script_download_data traffic ./tft_outputs

In [ ]:
# !chmod 777 run.sh
# !./run.sh
# !rm -r /content/drive/Shareddrives/CS726-Advanced-Machine-Learning/tft_tf2-master/tft_outputs/saved_models/traffic/fixed

In [ ]:
# !python3 script_train_fixed_params.py "traffic" "./tft_outputs" "no" "yes"

In [3]:
from script_train_fixed_params import main
import os

import data_formatters.electricity
import data_formatters.favorita
import data_formatters.traffic
import data_formatters.volatility

import argparse
import datetime as dte
import os

import data_formatters.base
import expt_settings.configs
import libs.hyperparam_opt
import libs.tft_model
import libs.utils as utils
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf

ExperimentConfig = expt_settings.configs.ExperimentConfig
HyperparamOptManager = libs.hyperparam_opt.HyperparamOptManager
ModelClass = libs.tft_model.TemporalFusionTransformer
tf.experimental.output_all_intermediates(True)


expt_name = "traffic"
use_gpu = True
model_folder = "/content/gdrive/Shareddrives/CS726-Advanced-Machine-Learning/tft_outputs/saved_models/traffic/fixed"
data_csv_path = "/content/gdrive/Shareddrives/CS726-Advanced-Machine-Learning/tft_outputs/data/traffic/hourly_data.csv"
data_formatter = data_formatters.traffic.TrafficFormatter()
use_testing_mode = False

num_repeats = 1

if not isinstance(data_formatter, data_formatters.base.GenericDataFormatter):
    raise ValueError(
        "Data formatters should inherit from" +
        "AbstractDataFormatter! Type={}".format(type(data_formatter)))

# Tensorflow setup
default_keras_session = tf.keras.backend.get_session()

if use_gpu:
    tf_config = utils.get_default_tensorflow_config(tf_device="gpu", gpu_id=0)

else:
    tf_config = utils.get_default_tensorflow_config(tf_device="cpu")

print("*** Training from defined parameters for {} ***".format(expt_name))

print("Loading & splitting data...")
raw_data = pd.read_csv(data_csv_path, index_col=0)
train, valid, test = data_formatter.split_data(raw_data)
train_samples, valid_samples = data_formatter.get_num_samples_for_calibration()
print("reached here")
# Sets up default params
fixed_params = data_formatter.get_experiment_params()
params = data_formatter.get_default_model_params()
params["model_folder"] = model_folder

# Parameter overrides for testing only! Small sizes used to speed up script.
train_samples, valid_samples = 20000, 2000
if use_testing_mode:
    fixed_params["num_epochs"] = 1
    params["hidden_layer_size"] = 6
    train_samples, valid_samples = 100, 10
# Sets up hyperparam manager
print("*** Loading hyperparm manager ***")
opt_manager = HyperparamOptManager({k: [params[k]] for k in params},
                                    fixed_params, model_folder)

# Training -- one iteration only
print("*** Running calibration ***")
print("Params Selected:")
print(fixed_params)

for k in params:
    print("{}: {}".format(k, params[k]))

print(train_samples)

best_loss = np.Inf
for _ in range(num_repeats):

    tf.reset_default_graph()
    with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

        tf.keras.backend.set_session(sess)

        params = opt_manager.get_next_parameters()
        model = ModelClass(params, use_cudnn=use_gpu)
        if not model.training_data_cached():
            model.cache_batched_data(train, "train", num_samples=train_samples)
            model.cache_batched_data(valid, "valid", num_samples=valid_samples)

        sess.run(tf.global_variables_initializer())
        model.fit()

        val_loss = model.evaluate()

        if val_loss < best_loss:
            opt_manager.update_score(params, val_loss, model)
            best_loss = val_loss

        tf.keras.backend.set_session(default_keras_session)

Selecting GPU ID=0
*** Training from defined parameters for traffic ***
Loading & splitting data...
Formatting train-valid-test splits.
Setting scalers with training data...
reached here
*** Loading hyperparm manager ***
*** Running calibration ***
Params Selected:
{'total_time_steps': 192, 'num_encoder_steps': 168, 'num_epochs': 100, 'early_stopping_patience': 5, 'multiprocessing_workers': 6, 'column_definition': [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)], 'input_size': 5, 'output_size': 1, 'category_counts': [963], 'input_obs_loc': [0], 

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


20000/20000 [==============================] - 280s 14ms/sample - loss: 3.1962 - val_loss: 1.6782
Epoch 2/100
20000/20000 [==============================] - 254s 13ms/sample - loss: 0.8298 - val_loss: 0.9471
Epoch 3/100
20000/20000 [==============================] - 254s 13ms/sample - loss: 0.7767 - val_loss: 0.4720
Epoch 4/100
20000/20000 [==============================] - 252s 13ms/sample - loss: 0.5628 - val_loss: 0.7090
Epoch 5/100
20000/20000 [==============================] - 252s 13ms/sample - loss: 0.5865 - val_loss: 0.5814
Epoch 6/100
20000/20000 [==============================] - 254s 13ms/sample - loss: 0.5081 - val_loss: 0.4125
Epoch 7/100
20000/20000 [==============================] - 253s 13ms/sample - loss: 0.4596 - val_loss: 0.4224
Epoch 8/100
20000/20000 [==============================] - 254s 13ms/sample - loss: 0.4314 - val_loss: 0.3074
Epoch 9/100
20000/20000 [==============================] - 253s 13ms/sample - loss: 0.4151 - val_loss: 0.3363
Epoch 10/100
20000/200

In [ ]:
print("*** Running tests ***")
# import data_formatters.electricity
# import data_formatters.favorita
# import data_formatters.traffic
# import data_formatters.volatility

# import argparse
# import datetime as dte
# import os

# import data_formatters.base

tf.reset_default_graph()
with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:
    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)
    print("Validation loss is : ", val_loss)
    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    print(type(output_map))
    print(output_map.keys())
    targets = data_formatter.format_predictions(output_map["targets"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p50_loss = utils.numpy_normalised_quantile_loss(
        extract_numerical_data(targets), extract_numerical_data(p50_forecast),
        0.5)
    p90_loss = utils.numpy_normalised_quantile_loss(
        extract_numerical_data(targets), extract_numerical_data(p90_forecast),
        0.9)

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])
print()
print("Normalised Quantile Loss for Test Data: P50={}, P90={}".format(
    p50_loss.mean(), p90_loss.mean()))

Streaming output truncated to the last 5000 lines.
  3.3932498e-03  3.0691877e-03 -6.3277166e-03 -4.6559544e-03
  7.3531978e-03 -4.1172123e-03  3.3126158e-05  4.5037926e-03
  1.0159329e-03 -3.9774482e-03  1.1615236e-03  4.5706183e-03
  1.4842444e-03  1.3087591e-02 -4.2226240e-03  3.4393000e-03
  2.2211879e-04 -8.1211226e-03  2.7965682e-03 -6.5974402e-03
 -6.3887099e-03 -6.3809347e-03 -6.1487551e-03  5.4779728e-03
 -3.0888561e-03  3.5898846e-03 -2.6952340e-03  2.1334060e-03
  5.3137843e-03 -2.5692341e-04  5.0341192e-04  8.2192980e-03
 -3.6108040e-03  3.3082133e-03  1.3338285e-03 -2.0879260e-03
  1.1814957e-04  5.9537120e-05  1.1119281e-02 -8.4830762e-04
  6.2331334e-03  5.7857484e-03 -3.3283336e-03  8.1244055e-03
  2.3920668e-04 -1.0186741e-02 -1.3318396e-03 -9.3382010e-03
 -9.6822074e-03  1.4812868e-03 -6.0751048e-05 -6.2968489e-03
 -4.9994059e-04 -5.9006386e-03 -6.0950904e-03 -4.6820245e-03
 -6.7842618e-04  1.0521900e-03 -8.4295627e-03  5.2390373e-03
  1.0024798e-03 -2.0571682e-03 -4.